# Clustering Crypto

In [129]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json
from pathlib import Path

### Fetching Cryptocurrency Data

In [130]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [131]:
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [132]:
# Create a DataFrame 

crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [133]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

Headers = list(crypto_df.columns)



In [134]:
new_columns = [ 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']

In [135]:
crypto_df =crypto_df[new_columns]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN
...,...,...,...,...,...,...
ARK,ARK,DPoS,True,DPoS,158681200.0,-1
ACT,Achain,DPoS,True,DPoS,1000000000,0
BLOCKSTAMP,BlockStamp,SHA-256,True,PoW,NaN,NaN
BST,Beshare Token,N/A,True,N/A,10000000000,-1


In [136]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0


In [137]:
# Keep only cryptocurrencies with a working algorithm

crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0


In [138]:
# Remove the "IsTrading" column

crypto_df = crypto_df.drop(columns= 'IsTrading')
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [139]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [140]:
# Remove rows with cryptocurrencies having no coins mined

crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3327130.86882,-1
XCP,CounterParty,SHA-256,PoW,2614878.378379,-1
NSR,NuShares,PoS,PoS,6161876092.0671,0
MONA,MonaCoin,Scrypt,PoW,82413387.471579,-1
TRI,Triangles Coin,X13,PoW/PoS,181125.443371,0


In [141]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[crypto_df['MaxSupply'] != 'N/A']
crypto_df = crypto_df.loc[crypto_df['ProofType'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3327130.86882,-1
XCP,CounterParty,SHA-256,PoW,2614878.378379,-1
NSR,NuShares,PoS,PoS,6161876092.0671,0
MONA,MonaCoin,Scrypt,PoW,82413387.471579,-1
TRI,Triangles Coin,X13,PoW/PoS,181125.443371,0


In [142]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

CoinName = crypto_df

In [143]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm

crypto_df = crypto_df.drop(columns= 'CoinName')

In [144]:
# Create dummy variables for text features

crypto_encoded = pd.get_dummies(data=crypto_df, columns=[ 'Algorithm', 'ProofType'])

In [145]:
# Standardize data
crypto_df_std = StandardScaler().fit_transform(crypto_encoded)
crypto_df_std

array([[-0.17245605, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696],
       [-0.17246168, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696],
       [-0.12383558, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696],
       ...,
       [-0.17115491, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696],
       [-0.17122957, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696],
       [-0.16458753, -0.13285415, -0.09016696, ..., -0.09016696,
        -0.09016696, -0.09016696]])

### Reducing Dimensions Using PCA

In [146]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_std)

In [147]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pca_df.head()

,PC 1,PC 2,PC 3
NVC,-1.215541,-0.638685,-0.255735
XCP,1.719250,-0.260389,-0.020896
NSR,-1.475145,0.157801,-0.547334
MONA,0.714617,-0.740452,-0.193213
TRI,-1.958795,-0.486181,-0.287454


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [148]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot


elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")


C:\Users\Alema\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [149]:
# Initialize the K-Means model

kmodel = KMeans(n_clusters=6, random_state=0)

# Fit the model

kmodel.fit(pca_df)

# Predict clusters

predictions = kmodel.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features

cyrpto_df_cluster = pd.DataFrame({
    "CoinName": CoinName.CoinName,
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.MaxSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "Class": kmodel.labels_,
    "Predictions": predictions, 
    },
)
cyrpto_df_cluster.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,Predictions
NVC,NovaCoin,Scrypt,PoW/PoS,3327130.86882,-1,-1.215541,-0.638685,-0.255735,1,1
XCP,CounterParty,SHA-256,PoW,2614878.378379,-1,1.719250,-0.260389,-0.020896,0,0
NSR,NuShares,PoS,PoS,6161876092.0671,0,-1.475145,0.157801,-0.547334,1,1
MONA,MonaCoin,Scrypt,PoW,82413387.471579,-1,0.714617,-0.740452,-0.193213,0,0
TRI,Triangles Coin,X13,PoW/PoS,181125.443371,0,-1.958795,-0.486181,-0.287454,1,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [150]:
# Create a 3D-Scatter with the PCA data and the clusters
coin_fig = px.scatter_3d(
    cyrpto_df_cluster,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
coin_fig.update_layout(legend=dict(x=0, y=1))
coin_fig.show()

#### Table of Tradable Cryptocurrencies

In [151]:
# Table with tradable cryptos

columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
cyrpto_df_cluster.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [152]:
# Scale data to create the scatter plot

cyrpto_df_cluster['TotalCoinsMined'] = cyrpto_df_cluster['TotalCoinsMined'].astype(float) / 1000000000
cyrpto_df_cluster['TotalCoinSupply'] = cyrpto_df_cluster['TotalCoinSupply'].astype(float) / 1000000000

In [154]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

cyrpto_df_cluster.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="Inferno_r", 
    hover_cols=['CoinName'])

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)